## Import Required Libraries and Dependencies

In [1]:
import tensorflow as tf
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
import os

2024-11-11 00:40:20.657841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 00:40:20.675987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 00:40:20.683411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 00:40:20.699577: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Set Parameters

In [2]:
img_height, img_width = 224, 224
batch_size = 32
num_classes = 2     # Cats and Dogs

## Load the cats abd dogs dataset

In [3]:
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_file = tf.keras.utils.get_file("cat_and_dogs_filtered.zip", dataset_url, extract=True)
base_dir = os.path.join(os.path.dirname(zip_file), 'cats_and_dogs_filtered')

## Set path for train and validation directories

In [4]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

## Load the pre-trained VGG16 model without the top layer

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

I0000 00:00:1731265824.315787   91911 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731265824.460890   91911 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731265824.464597   91911 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731265824.469026   91911 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

## Freeze the layers in the base model to retain pre-trained weights

In [6]:
for layer in base_model.layers:
    layer.trainable = False

## Add custom layers on top of the base model for our specific classification task

In [7]:
x = base_model.output
x = Flatten()(x)   # Flatten the output to 1D
x = Dense(512, activation='relu')(x)    # Add a dense layer with 512 units
x = Dropout(0.5)(x)     # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

## Combine base models and custom layers

In [8]:
model = Model(inputs = base_model.input, outputs = predictions)

## Compile the model

In [9]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Prepare the data with ImageDataGenerator for data augumentation

In [10]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Train the model

In [13]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

## Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

## Save the model

In [ ]:
model.save =("VGG16_transfer_learning_cats_dogs.h5")